In [6]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline, AutoConfig
import shap

ModuleNotFoundError: No module named 'shap'

In [ ]:
def load_pipeline(INPUT_PATH):
    model = AutoModelForSequenceClassification.from_pretrained(INPUT_PATH)
    tokenizer = AutoTokenizer.from_pretrained(INPUT_PATH)
    config = AutoConfig.from_pretrained(INPUT_PATH)
    p = pipeline('text-classification', model=model, tokenizer=tokenizer, config=config, return_all_scores=True)
    return p

In [ ]:
INPUT_PATH = '../../models/title'

pipeline_title = load_pipeline(INPUT_PATH)

In [ ]:
INPUT_PATH = '../../models/title_summary'

pipeline_title_summary = load_pipeline(INPUT_PATH)

In [ ]:
# Features pushing the prediction higher are shown in red, those pushing the prediction lower are in blue.

# LABEL_0--> CONTROVERSY, LABEL_1 --> NO_CONTROVERSY

In [ ]:
def try_input(pipeline, INPUT_TEXT, display=True):
    print(pipeline([INPUT_TEXT]))
    explainer = shap.Explainer(pipeline)
    shap_values = explainer([INPUT_TEXT])
    shap_values.output_names = ['CONTROVERSY', 'NO_CONTROVERSY']
    html_out = shap.plots.text(shap_values[0, :, 'CONTROVERSY'], display = display) 
    if not display:
        file = open("../../output/visuals/"+INPUT_TEXT[:10].replace(' ', '_').lower()+".html","w")
        file.write(html_out)
        file.close()
    #return shap_values
    

In [ ]:
INPUT_TEXT = "Guerra de Ucrania: Basta ya de mentiras, medias verdades y desinformación" # -- for adding the summary

html_out = try_input(pipeline_title, INPUT_TEXT)

In [ ]:
INPUT_TEXT = "Guerra de Ucrania: Basta ya de mentiras, medias verdades y desinformación -- El engaño de hoy será el lamento de mañana. Por ello, no solo hay que describir lo que está sucediendo desde múltiples puntos de vista, sino que también es necesario intentar alcanzar las razones por las que acontece. No es sencillo y, seguramente, no lo consiga debido a las múltiples limitaciones que poseo, pero es lo que he intentado hacer. Este es el resultado en quince puntos. No es una lectura cómoda ni corta, pero creo que necesaria. " # -- for adding the summary

try_input(pipeline_title_summary, INPUT_TEXT)

In [ ]:
# TRY THE FIRST 4 INSTANCES IN WHICH T_S DOES IT RIGHT

In [ ]:
import json

analysis_data = []
with open('analysis_data.json', 'r') as file:
    for line in file.readlines():
        d = json.loads(line)
        analysis_data.append(d)

In [ ]:
for i,data in enumerate(analysis_data):
    if data['controversy'] and data['answer_summary'] == 'CONTROVERSY':
        print(i)

### Ejemplo 1

In [ ]:
data = analysis_data[1]
print(data['answer_title'], data['answer_summary'])

In [ ]:
try_input(pipeline_title, "Los vecinos de Chamartín rodean y plan tan cara a los ultras de las cacerolas")

In [ ]:
try_input(pipeline_title_summary, data['title']+' -- '+data['content'])

In [ ]:
data2 = analysis_data[23]
print(data2['answer_title'], data2['answer_summary'])

In [ ]:
try_input(pipeline_title, data2['title'], display = True)

In [ ]:
try_input(pipeline_title_summary, data2['title']+' -- '+data2['content'])

In [ ]:
data3 = analysis_data[36]
print(data3['answer_title'], data3['answer_summary'])

In [ ]:
try_input(pipeline_title, data3['title'], display=True)

In [ ]:
try_input(pipeline_title_summary, data3['title']+' -- '+data3['content'])

In [ ]:
data4 = analysis_data[53]
print(data4['answer_title'], data4['answer_summary'])

In [ ]:
try_input(pipeline_title, data4['title'])

In [ ]:
try_input(pipeline_title_summary, data4['title']+' -- '+data4['content'])